## A. Upload CSVs to S3 (using boto3)

In [1]:
# install dependencies or put them in a requirements.txt file

# !pip install sqlalchemy
# !pip install sqlalchemy psycopg2-binary pandas


In [1]:
pip install boto3 psycopg2-binary python-dotenv pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
import os
import boto3

# Explicitly specify the path to the aws.env file
load_dotenv(dotenv_path="c:/Users/kaoth/OneDrive/Desktop/Data Science Lectures/Machine Learning/Amdari Internship/medoptix-ai-internship/aws.env")

# Access the AWS credentials
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
BUCKET_NAME = os.getenv("S3_BUCKET") 


# Initialize the S3 client using the credentials
s3 = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

print("AWS S3 client initialized successfully.")

AWS S3 client initialized successfully.


## B. Load from S3 → PostgreSQL


In [3]:
!pip install sqlalchemy



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Connect to Postgres

In [4]:
from dotenv import load_dotenv
import os
import psycopg2

load_dotenv(dotenv_path="C:/Users/kaoth/OneDrive/Desktop/Data Science Lectures/Machine Learning/Amdari Internship/medoptix-ai-internship/aiven.env")

conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD")
)
cursor = conn.cursor()
print("✅ Connected to PostgreSQL.")


✅ Connected to PostgreSQL.


# Data Modelling & Defining Foreign Key relationships


- patients (PK: patient_id)
- sessions (PK: session_id, FK: patient_id)
- feedback (PK: feedback_id, FK: session_id)


TASK 1 - Model and Define the relationship for the remaining set of dataset
 - clinics
 - dropout_flags
 - intervention.csv

TASK 2 - Create Schema for these dataset and upload then into postgres

## C. Upload Data → PostgreSQL

####  Create Tables with Foreign Key Constraints

In [6]:
import pandas as pd
import os

base_path = "C:/Users/kaoth/OneDrive/Desktop/Data Science Lectures/Machine Learning/Amdari Internship/Project_6/medoptix_data"

clinics = pd.read_csv(os.path.join(base_path, "clinics.csv"))
patients = pd.read_csv(os.path.join(base_path, "patients.csv"))
sessions = pd.read_csv(os.path.join(base_path, "sessions.csv"))
feedback = pd.read_csv(os.path.join(base_path, "feedback.csv"))
interventions = pd.read_csv(os.path.join(base_path, "interventions.csv"))
dropout_flags = pd.read_csv(os.path.join(base_path, "dropout_flags.csv"))


In [7]:
dfs = {
    "clinics": clinics,
    "patients": patients,
    "sessions": sessions,
    "feedback": feedback,
    "interventions": interventions,
    "dropout_flags": dropout_flags
}

for name, df in dfs.items():
    print(f"\n{name.upper()} column types:")
    print(df.dtypes)



CLINICS column types:
clinic_id        int64
city            object
country         object
type            object
postcode         int64
capacity         int64
staff_count      int64
speciality      object
avg_rating     float64
dtype: object

PATIENTS column types:
patient_id           int64
age                  int64
gender              object
bmi                float64
smoker                bool
chronic_cond        object
injury_type         object
signup_date         object
referral_source     object
consent               bool
clinic_id            int64
insurance_type      object
dtype: object

SESSIONS column types:
session_id           object
patient_id            int64
date                 object
week                  int64
duration              int64
pain_level            int64
exercise_type        object
home_adherence_pc     int64
satisfaction          int64
therapist_id          int64
dtype: object

FEEDBACK column types:
feedback_id     object
session_id      object
commen

### Load Data and Insert into Tables

In [5]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path=r"C:/Users/kaoth/.../aiven.env")

bucket = os.getenv("S3_BUCKET")

print("Bucket:", bucket)  # This should print: medoptix-bucket


Bucket: medoptix-bucket


In [6]:
import pandas as pd
from io import StringIO

# Function to read CSV from S3
def read_csv_from_s3(filename):
    response = s3.get_object(Bucket=bucket, Key=filename)
    return pd.read_csv(StringIO(response['Body'].read().decode('utf-8')))

# Function to insert DataFrame into PostgreSQL
def insert_dataframe(df, table_name):
    df = df.where(pd.notnull(df), None)  # Replace NaN with None
    inserted = 0
    for i, row in df.iterrows():
        cols = ','.join(row.index)
        vals = ','.join(['%s'] * len(row))
        sql = f"INSERT INTO {table_name} ({cols}) VALUES ({vals})"
        try:
            cursor.execute(sql, tuple(row))
            inserted += 1
        except Exception as e:
            print(f"❌ Error inserting row {i} into '{table_name}': {e}")
            print("➡️ Row data:", row.to_dict())
            conn.rollback()
            break
    conn.commit()
    print(f"✅ Inserted {inserted} rows into '{table_name}'.")

# File-to-table mapping
file_table_map = {
    "clinics.csv": "clinics",
    "patients.csv": "patients",
    "sessions.csv": "sessions",
    "feedback.csv": "feedback",
    "interventions.csv": "interventions",
    "dropout_flags.csv": "dropout_flags"
}

# Insert all data
for filename, table in file_table_map.items():
    print(f"\n📥 Inserting {filename} → {table}")
    df = read_csv_from_s3(filename)
    insert_dataframe(df, table)

print("✅ All data inserted successfully.")



📥 Inserting clinics.csv → clinics
❌ Error inserting row 0 into 'clinics': duplicate key value violates unique constraint "clinics_pkey"
DETAIL:  Key (clinic_id)=(1) already exists.

➡️ Row data: {'clinic_id': 1, 'city': 'London', 'country': 'UK', 'type': 'NHS', 'postcode': 1184, 'capacity': 109, 'staff_count': 26, 'speciality': 'Neuro-rehab', 'avg_rating': 4.7}
✅ Inserted 0 rows into 'clinics'.

📥 Inserting patients.csv → patients
❌ Error inserting row 0 into 'patients': duplicate key value violates unique constraint "patients_pkey"
DETAIL:  Key (patient_id)=(1) already exists.

➡️ Row data: {'patient_id': 1, 'age': 29, 'gender': 'Male', 'bmi': 23.9, 'smoker': False, 'chronic_cond': None, 'injury_type': 'Knee', 'signup_date': '2025-04-27 00:12:44.286283', 'referral_source': 'Insurance', 'consent': True, 'clinic_id': 6, 'insurance_type': 'Public'}
✅ Inserted 0 rows into 'patients'.

📥 Inserting sessions.csv → sessions
❌ Error inserting row 0 into 'sessions': duplicate key value violate

In [9]:
tables = ["clinics", "patients", "sessions", "feedback", "interventions", "dropout_flags"]

for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table};")
    count = cursor.fetchone()[0]
    print(f"📊 {table}: {count} rows")


📊 clinics: 8 rows
📊 patients: 5000 rows
📊 sessions: 70236 rows
📊 feedback: 49165 rows
📊 interventions: 5016 rows
📊 dropout_flags: 5000 rows


In [10]:
# Assuming you have this function already defined
def read_csv_from_s3(filename):
    response = s3.get_object(Bucket=bucket, Key=filename)
    return pd.read_csv(StringIO(response['Body'].read().decode('utf-8')))

# Compare row counts
for file, table in file_table_map.items():
    df = read_csv_from_s3(file)
    cursor.execute(f"SELECT COUNT(*) FROM {table};")
    db_count = cursor.fetchone()[0]
    print(f"✅ {table}: {db_count} in DB vs {len(df)} in S3")


✅ clinics: 8 in DB vs 8 in S3
✅ patients: 5000 in DB vs 5000 in S3
✅ sessions: 70236 in DB vs 70236 in S3
✅ feedback: 49165 in DB vs 49165 in S3
✅ interventions: 5016 in DB vs 5016 in S3
✅ dropout_flags: 5000 in DB vs 5000 in S3


## D. Read Data → PostgreSQL - (Prepare data for EDA)


In [7]:
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

def get_db_engine():
    """Create and return a SQLAlchemy engine using env variables"""
    db_url = (
        f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@"
        f"{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
    )
    return create_engine(db_url)

# ✅ Initialize engine
engine = get_db_engine()

# ✅ Define your queries
patients_query = "SELECT * FROM patients"
sessions_query = "SELECT * FROM sessions"
feedback_query = "SELECT * FROM feedback"
clinics_query = "SELECT * FROM clinics"
interventions_query = "SELECT * FROM interventions"
dropout_query = "SELECT * FROM dropout_flags"

# ✅ Load data from PostgreSQL into DataFrames
patients_df = pd.read_sql(patients_query, engine)
sessions_df = pd.read_sql(sessions_query, engine)
feedback_df = pd.read_sql(feedback_query, engine)
clinics_df = pd.read_sql(clinics_query, engine)
interventions_df = pd.read_sql(interventions_query, engine)
dropout_df = pd.read_sql(dropout_query, engine)

# ✅ Quick check
print("Patients:", patients_df.shape)
print("Sessions:", sessions_df.shape)
print("Feedback:", feedback_df.shape)
print("Clinics:", clinics_df.shape)
print("Interventions:", interventions_df.shape)
print("Dropout Flags:", dropout_df.shape)


Patients: (5000, 12)
Sessions: (70236, 10)
Feedback: (49165, 4)
Clinics: (8, 9)
Interventions: (5016, 6)
Dropout Flags: (5000, 3)


In [9]:
# Save data

save_path = r"C:\Users\kaoth\OneDrive\Desktop\Data Science Lectures\Machine Learning\Amdari Internship\medoptix-ai-internship\medoptix_data\processed"

patients_df.to_csv(f"{save_path}\\patients.csv", index=False)
sessions_df.to_csv(f"{save_path}\\sessions.csv", index=False)
feedback_df.to_csv(f"{save_path}\\feedback.csv", index=False)
clinics_df.to_csv(f"{save_path}\\clinics.csv", index=False)
interventions_df.to_csv(f"{save_path}\\interventions.csv", index=False)
dropout_df.to_csv(f"{save_path}\\dropout_flags.csv", index=False)


### D1- Display basic info and summary statistics for each table

In [12]:
# Patients
print("Patients Data Overview")
patients_df.head(2)
patients_df.info()
patients_df.describe()



Patients Data Overview
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   patient_id       5000 non-null   int64         
 1   age              5000 non-null   int64         
 2   gender           5000 non-null   object        
 3   bmi              5000 non-null   float64       
 4   smoker           5000 non-null   bool          
 5   chronic_cond     1931 non-null   object        
 6   injury_type      5000 non-null   object        
 7   signup_date      5000 non-null   datetime64[ns]
 8   referral_source  5000 non-null   object        
 9   consent          5000 non-null   bool          
 10  clinic_id        5000 non-null   int64         
 11  insurance_type   5000 non-null   object        
dtypes: bool(2), datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 400.5+ KB


,patient_id,age,bmi,signup_date,clinic_id
count,5000.000000,5000.000000,5000.000000,5000,5000.000000
mean,2500.500000,50.907200,25.174580,2024-12-10 09:48:44.286283520,3.387800
min,1.000000,18.000000,17.000000,2024-06-13 00:12:44.286283,1.000000
25%,1250.750000,34.000000,22.400000,2024-09-08 00:12:44.286283008,2.000000
50%,2500.500000,51.000000,25.100000,2024-12-10 12:12:44.286283008,3.000000
75%,3750.250000,68.000000,27.700000,2025-03-11 00:12:44.286283008,5.000000
max,5000.000000,84.000000,38.800000,2025-06-12 00:12:44.286283,8.000000
std,1443.520003,19.403203,3.769633,NaN,1.749521


In [13]:
# Sessions
print("Sessions Data Overview")
sessions_df.head(2)
sessions_df.info()
sessions_df.describe()


Sessions Data Overview
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70236 entries, 0 to 70235
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   session_id         70236 non-null  object 
 1   patient_id         70236 non-null  int64  
 2   date               70236 non-null  object 
 3   week               70236 non-null  int64  
 4   duration           70236 non-null  int64  
 5   pain_level         70236 non-null  int64  
 6   exercise_type      70236 non-null  object 
 7   home_adherence_pc  70236 non-null  float64
 8   satisfaction       70236 non-null  int64  
 9   therapist_id       70236 non-null  int64  
dtypes: float64(1), int64(6), object(3)
memory usage: 5.4+ MB


,patient_id,week,duration,pain_level,home_adherence_pc,satisfaction,therapist_id
count,70236.000000,70236.000000,70236.000000,70236.000000,70236.000000,70236.000000,70236.000000
mean,2498.827652,6.490717,39.511134,1.982317,69.333775,6.490831,1099.411498
std,1444.925172,3.451124,11.516467,1.484108,14.846721,1.236411,57.610261
min,1.000000,1.000000,20.000000,1.000000,0.000000,2.000000,1000.000000
25%,1247.000000,3.000000,30.000000,1.000000,59.000000,6.000000,1049.000000
50%,2499.000000,6.000000,40.000000,1.000000,69.000000,6.000000,1099.000000
75%,3751.000000,9.000000,49.000000,3.000000,80.000000,7.000000,1149.000000
max,5000.000000,12.000000,59.000000,8.000000,100.000000,10.000000,1199.000000


In [14]:
# Feedback
print("Feedback Data Overview")
feedback_df.head()
feedback_df.info()
feedback_df.describe()

Feedback Data Overview
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49165 entries, 0 to 49164
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   feedback_id  49165 non-null  object 
 1   session_id   49165 non-null  object 
 2   comments     49165 non-null  object 
 3   sentiment    49165 non-null  float64
dtypes: float64(1), object(3)
memory usage: 1.5+ MB


,sentiment
count,49165.000000
mean,0.304360
std,0.356693
min,-0.440000
25%,0.000000
50%,0.459000
75%,0.542000
max,0.802000


In [15]:
# clinics
print("clinics Data Overview")
clinics_df.head()
clinics_df.info()
clinics_df.describe()

clinics Data Overview
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   clinic_id    8 non-null      int64  
 1   city         8 non-null      object 
 2   country      8 non-null      object 
 3   type         8 non-null      object 
 4   postcode     8 non-null      object 
 5   capacity     8 non-null      int64  
 6   staff_count  8 non-null      int64  
 7   speciality   8 non-null      object 
 8   avg_rating   8 non-null      float64
dtypes: float64(1), int64(3), object(5)
memory usage: 708.0+ bytes


,clinic_id,capacity,staff_count,avg_rating
count,8.00000,8.000000,8.000000,8.000000
mean,4.50000,153.250000,21.500000,4.237500
std,2.44949,24.574958,5.976143,0.373927
min,1.00000,109.000000,13.000000,3.700000
25%,2.75000,144.750000,18.250000,4.075000
50%,4.50000,151.500000,20.000000,4.300000
75%,6.25000,161.000000,26.500000,4.450000
max,8.00000,193.000000,30.000000,4.700000


In [16]:
# interventions
print("interventions Data Overview")
interventions_df.head()
interventions_df.info()
interventions_df.describe()

interventions Data Overview
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5016 entries, 0 to 5015
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   intervention_id  5016 non-null   object        
 1   patient_id       5016 non-null   int64         
 2   sent_at          5016 non-null   datetime64[ns]
 3   channel          5016 non-null   object        
 4   message          5016 non-null   object        
 5   responded        5016 non-null   bool          
dtypes: bool(1), datetime64[ns](1), int64(1), object(3)
memory usage: 201.0+ KB


,patient_id,sent_at
count,5016.000000,5016
mean,2530.523525,2025-01-04 15:20:46.582934016
min,2.000000,2024-06-17 00:12:44.286283
25%,1259.750000,2024-10-03 00:12:44.286283008
50%,2517.000000,2025-01-03 00:12:44.286283008
75%,3818.000000,2025-04-07 00:12:44.286283008
max,5000.000000,2025-07-26 00:12:44.286283
std,1457.936905,NaN


In [17]:
# dropout
print("dropout Data Overview")
dropout_df.head()
dropout_df.info()
dropout_df.describe()

dropout Data Overview
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   patient_id    5000 non-null   int64  
 1   dropout       5000 non-null   bool   
 2   dropout_week  81 non-null     float64
dtypes: bool(1), float64(1), int64(1)
memory usage: 83.1 KB


,patient_id,dropout_week
count,5000.000000,81.000000
mean,2500.500000,9.481481
std,1443.520003,0.760117
min,1.000000,7.000000
25%,1250.750000,9.000000
50%,2500.500000,10.000000
75%,3750.250000,10.000000
max,5000.000000,10.000000


## E. EDA → (Exploratory Data Analysis)

Steps in EDA
 - Data Exploration
    - Head()
    - info()
    - describe()
 - Data Cleaning 
    - Identifying & Handling Missing Values
    - Identifying & Handling Duplicates
 - Data Transformation (Feature Engineering)
   - Standardizing data types (text, date, replacing values)
   - Log Transform (for highly skewed data e.g., income, prices)
   - Binning (Convert continuous variables into categorical bins e.g., age ranges)
   - Encoding (Convert categorical variables into numerical format (e.g., One-Hot, Label Encoding))
 - Data Distributions
    - Histograms for distributions of numerical features (age, bmi etc)- Also to check if the features are normally distributed or skewed (e.g., pain level or age should be somewhat evenly distributed just like we discovered in the last drop-in session).
    - Boxplots to detect outliers & handle them.
    - Pairplots to visualize relationships between multiple features.
 -  Correlation Analysis (dentify relationships between variables that may help in predictive modeling)
    - Correlation Matrix to understand feature dependencies.
    - Heatmap for visual representation of correlations.
 - Understanding Dropout Behavior
    - investigate the correlation between features and patient dropout.



#### E-2: Data Cleaning (Handling Missing Values)

In [18]:
#5. Check for Missing Values in All Tables
print("\nMissing Values in Patients Data:")
print(patients_df.isnull().sum())

print("\nMissing Values in Sessions Data:")
print(sessions_df.isnull().sum())

print("\nMissing Values in Feedback Data:")
print(feedback_df.isnull().sum())




Missing Values in Patients Data:
patient_id            0
age                   0
gender                0
bmi                   0
smoker                0
chronic_cond       3069
injury_type           0
signup_date           0
referral_source       0
consent               0
clinic_id             0
insurance_type        0
dtype: int64

Missing Values in Sessions Data:
session_id           0
patient_id           0
date                 0
week                 0
duration             0
pain_level           0
exercise_type        0
home_adherence_pc    0
satisfaction         0
therapist_id         0
dtype: int64

Missing Values in Feedback Data:
feedback_id    0
session_id     0
comments       0
sentiment      0
dtype: int64


In [19]:
# 6. Clean Missing Values
# Example of filling missing values for categorical and numerical data
patients_df['chronic_cond'] = patients_df['chronic_cond'].fillna(patients_df['chronic_cond'].mode()[0])

In [20]:
# check if null values has been properly handled
patients_df.isnull().sum()

patient_id         0
age                0
gender             0
bmi                0
smoker             0
chronic_cond       0
injury_type        0
signup_date        0
referral_source    0
consent            0
clinic_id          0
insurance_type     0
dtype: int64

In [21]:
print("\nMissing Values in Clinics Data:")
print(clinics_df.isnull().sum())

print("\nMissing Values in Interventions Data:")
print(interventions_df.isnull().sum())

print("\nMissing Values in Dropout Data:")
print(dropout_df.isnull().sum())


Missing Values in Clinics Data:
clinic_id      0
city           0
country        0
type           0
postcode       0
capacity       0
staff_count    0
speciality     0
avg_rating     0
dtype: int64

Missing Values in Interventions Data:
intervention_id    0
patient_id         0
sent_at            0
channel            0
message            0
responded          0
dtype: int64

Missing Values in Dropout Data:
patient_id         0
dropout            0
dropout_week    4919
dtype: int64


#### E-3 - TASK 3 (Data Cleaning)
- Identifying & Handling Duplicates


In [22]:
# Check for complete duplicate rows
dataframes = {
    "patients_df": patients_df,
    "sessions_df": sessions_df,
    "feedback_df": feedback_df,
    "clinics_df": clinics_df,
    "interventions_df": interventions_df,
    "dropout_df": dropout_df
}

for name, df in dataframes.items():
    duplicates = df[df.duplicated()]
    print(f"🔍 Found {len(duplicates)} complete duplicate rows in {name}")



🔍 Found 0 complete duplicate rows in patients_df
🔍 Found 0 complete duplicate rows in sessions_df
🔍 Found 0 complete duplicate rows in feedback_df
🔍 Found 0 complete duplicate rows in clinics_df
🔍 Found 0 complete duplicate rows in interventions_df
🔍 Found 0 complete duplicate rows in dropout_df
